In [1]:
import os

In [2]:
%pwd

'e:\\Python Projects\\Deep_learning\\mlops_test1\\research'

In [3]:
os.chdir("../")

In [4]:
from dataclasses import dataclass
from pathlib import Path
@dataclass
class PrepareCallbacksConfig:
    root_directory: Path
    tensorboard_root_log_directory: Path
    checkpoint_model_filepath: Path



Update the confiuration manager

In [5]:
from src.chicken_disease_classification.constants import CONFIG_FILE_PATH,PARAMS_FILE_PATH
from src.chicken_disease_classification.utils.common import read_yaml,create_directories


class ConfigurationManager():
    def __init__(self, config_file_path, params_file_path):
        self.config= read_yaml(config_file_path)
        self.params= read_yaml(params_file_path)
        create_directories([self.config.artifacts_root])

    def get_prepare_callbacks_config(self)-> PrepareCallbacksConfig :
        config= self.config.prepare_callbacks
        model_chkpt_dir= os.path.dirname(config.checkpoint_model_filepath)
        create_directories([Path(model_chkpt_dir),Path(config.tensorboard_root_log_dir)])

        prepare_callbacks_config = PrepareCallbacksConfig(
            root_directory=Path(config.root_directory),
            tensorboard_root_log_directory= Path(config.tensorboard_root_log_dir),
            checkpoint_model_filepath= Path(config.checkpoint_model_filepath)
        )
        return prepare_callbacks_config


Create the component which uses the PrepareCallbacksConfig object

In [6]:
import os
import urllib.request as request
from zipfile import ZipFile
import tensorflow as tf
import time

In [7]:
class PrepareCallbacks:
    def __init__(self, config:PrepareCallbacksConfig):
        self.config= config

    @property
    def _create_tb_callbacks(self):
        timestamp = time.strftime("%Y-%m-%d-%H-%M-%S")
        tb_running_log_dir = os.path.join(
            self.config.tensorboard_root_log_directory,
            f"tb_logs_at_{timestamp}",
        )
        return tf.keras.callbacks.TensorBoard(log_dir=tb_running_log_dir)
    

    @property
    def _create_ckpt_callbacks(self):
        return tf.keras.callbacks.ModelCheckpoint(
            filepath=self.config.checkpoint_model_filepath,
            save_best_only=True
        )


    def get_tb_ckpt_callbacks(self):
        return [
            self._create_tb_callbacks,
            self._create_ckpt_callbacks
        ]


        
    

In [8]:
# create the pipeline
from src.chicken_disease_classification.constants import CONFIG_FILE_PATH,PARAMS_FILE_PATH

In [9]:
try:
    config_manager= ConfigurationManager(config_file_path= CONFIG_FILE_PATH,params_file_path=PARAMS_FILE_PATH)
    prepare_callbacks_config= config_manager.get_prepare_callbacks_config()
    prepare_callback_component= PrepareCallbacks(config= prepare_callbacks_config)
    callback_list= prepare_callback_component.get_tb_ckpt_callbacks()
except Exception as e:
    raise e

2025-05-17 21:15:43,018 - INFO - yaml file: e:\Python Projects\Deep_learning\mlops_test1\config\config.yaml loaded successfully
2025-05-17 21:15:43,026 - INFO - yaml file: e:\Python Projects\Deep_learning\mlops_test1\params.yaml loaded successfully
2025-05-17 21:15:43,026 - INFO - created directory at: artifacts
2025-05-17 21:15:43,026 - INFO - created directory at: artifacts\prepare_callbacks\checkpoint_dir
2025-05-17 21:15:43,026 - INFO - created directory at: artifacts\prepare_callbacks\tensorboard_log_dir
